# Part 0: Dataloader and Visualizations

In [1]:
import numpy as np
from PIL import Image
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import wandb

from voc_dataset import VOCDataset
from utils import *

USE_WANDB = False

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Q0.1: Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself and we have completed this part for you in the starter code. 

Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf). Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals for you (you downloaded this in the data preparation step).

Your task is to change the dataloader to obtain the proposed bounding boxes for each image. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [2]:
dataset = VOCDataset('trainval', top_n=10)

INITIALIZED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1111111


**Q0.1**: Load the image corresponding to index 2020 and print the GT labels associated with it.

**Hint**: items at a particular index can be accesed by usual indexing notation (dataset[idx])

In [24]:
# TODO: get the image information from index 2020
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))
idx = 2020
d = dataset[2020]
img = d["image"]
labels = d["label"]
print("labels = ", labels)
label_name = class_id_to_label[int(np.argmax(labels))]
print("Label name = ", label_name)
img = tensor_to_PIL(img)
# img.show()

labels =  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0.])
Label name =  train


## Q0.2 and Q0.3: Wandb Logging
First, let's initialize a Weights and Biases project. 

In [4]:
USE_WANDB = True
if USE_WANDB:
    wandb.init(project="VLR-HW1-Task0", reinit=True)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: ndamis (ndamises). Use `wandb login --relogin` to force relogin


**Q0.2**: Complete this block for overlaying the ground truth box on an image.

**Hint**: convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions). You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [6]:

# TODO: load the GT information corresponding to index 2020.
original_image = img
gt_labels = d["gt_classes"]
gt_boxes = d["gt_boxes"]

img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_labels, gt_boxes),
        "class_labels": class_id_to_label,
    },
})

# TODO: log the GT bounding box
wandb.log({"Q- 0.02 - Ground Truth Bounding Box": img})    


**Q0.3**: Visualize the top 10 bounding proposals corresponding to index 2020.

**Hint**: Check the `get_box_data` function in `utils.py` and understand how it is being used. 

In [20]:
# TODO: plot top ten proposals (of bounding boxes)
rois = d["rois"]
max = np.argmax(labels)
print(max)
print("Roi shape ", rois.shape)
arr = [int(max)]*10
img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(arr, rois),
        "class_labels": class_id_to_label,
    },
})

wandb.log({"ROI Proposals": img})

tensor(18)
Roi shape  (10, 4)
